In [2]:
import matplotlib.pyplot as plt
import numpy as np

import pygame

pygame 2.5.0 (SDL 2.30.10, Python 3.11.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


<frozen importlib._bootstrap>:241: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


In [3]:
def RK4(fun, x, dt, t = 0, a = 0):

    k1 = fun(t,      x,         a)
    k2 = fun(t+dt/2, x+dt*k1/2, a)
    k3 = fun(t+dt/2, x+dt*k2/2, a)
    k4 = fun(t+dt,   x+dt*k3,   a)

    y = x + dt/6*(k1+2*k2+2*k3+k4)
    return y

In [25]:
l   = 200
g   = 980
gamma = .1
gammath = .1

def cart_evol(t, x, a = 0):
    vDot = a - gamma * x[1]
    xDot = x[1]
    return np.array([xDot, vDot])

def pend_evol(t, x, a = 0):
    thDotDot = (a * np.cos(x[0]) - g * np.sin(x[0]))/l - gammath * x[1]
    thDot    = x[1]
    return np.array([thDot, thDotDot])

def get_pos_pend(ang, point, L):
    return point[0] - L*np.sin(ang), point[1] + L*np.cos(ang)

def update_screen(screen, cart, th,dt):
    screen.fill((255,255,255))

    pend_pos  = get_pos_pend(th, cart.center, l)

    global target_H, score
    if target_H:
        pygame.draw.line(screen, (0,150,40), (0,target_H), (1600,target_H), 10)
        
        if pend_pos[1] < target_H:
            score += dt

    pygame.draw.line(screen, (40,40,40),
                    cart.center, pend_pos,
                    width = 10)
    pygame.draw.rect(screen, (130,10,10), cart)
    pygame.draw.circle(screen, (0,0,0), pend_pos, 10)

In [29]:
cart_size = [40,20]
cart_pos  = [400-20,300-10]

target_th = 1
target_H = get_pos_pend(target_th, cart_pos, l)[1]

th, thDot  = 0, 0
a, v, x = 0, 0, cart_pos[0]

score = 0

pygame.init()
pygame.font.init()

screen = pygame.display.set_mode((1500,600))
clock  = pygame.time.Clock()
player = pygame.Rect(cart_pos+cart_size)

my_font = pygame.font.SysFont('Comic Sans MS', 50)

pend_pos = get_pos_pend(th, player.center, l)
dt = clock.tick()

time = 0
while time < 20:
  
  time = pygame.time.get_ticks()/1000
  dt = clock.tick(120)/1000 # Por los milisecs
  x, v = RK4(cart_evol, [x, v], dt, a = a)
  th, thDot = RK4(pend_evol, [th, thDot], dt, a = a)
  player.move_ip(x-player.x, 0)

  update_screen(screen, player, th,dt)
  # time and fps'{:-9} YES votes  {:2.2%}'.format(yes_votes, percentage)
  screen.blit(my_font.render("TIME: %.1f"%(time), False, (255, 0, 0)), (100,100))
  screen.blit(my_font.render("SCORE: %.1f"%(score), False, (0,150,40)), (100,50))
  screen.blit(my_font.render("FPS: %i"%(clock.get_fps()), False, (255, 0, 0)), (100,150))

  key = pygame.key.get_pressed()
  if key[pygame.K_a] == True:
    a = -1000
  elif key[pygame.K_d] == True:
    a = 1000
  else:
    a = 0
  
  for event in pygame.event.get():
    if event.type == pygame.QUIT:
      pygame.quit()
      continue
  
  pygame.display.update()

pygame.quit()
print(score)

17.75799999999987


# Testing

In [ ]:
l = 1
g = 9.8

In [ ]:
def fun(t, x, a_ext = 0):
    thDotDot = (a_ext * np.cos(x[0]) - g * np.sin(x[0]))/l
    thDot    = x[1]
    return np.array([thDot, thDotDot])

def RK4(fun, x, dt, t = 0, a = 0):

    k1 = fun(t,      x,         a)
    k2 = fun(t+dt/2, x+dt*k1/2, a)
    k3 = fun(t+dt/2, x+dt*k2/2, a)
    k4 = fun(t+dt,   x+dt*k3,   a)

    y = x + dt/6*(k1+2*k2+2*k3+k4)
    return y

In [ ]:
delta_t = 1E-3
steps   = int(1e4)

x1 = np.zeros(steps)
x2 = np.zeros(steps)

x1[0], x2[0] = 1, 0

for step in range(1, steps):
    x1[step], x2[step] = RK4(cart_evol, [x1[step-1], x2[step-1]], delta_t, a = 20)

plt.plot(x1)
#plt.plot(x2)